Aluno: Arthur Mauricio
Lista 6b

In [1]:
import numpy as np

### Questão 1

- Ponto Fixo

In [23]:
def ponto_fixo(f, init, max_iter = 100, erro_tol = 1e-1):
    g = lambda x : f(x) + x
    last_root = init
    root = None
    for i in range(max_iter):
        root = g(last_root)
        if np.abs((root - last_root)/root) <= erro_tol:
            print("Valor dentro do treshold de erro")
            return root
        last_root = root
    return root

In [27]:
ponto_fixo(f = lambda x : np.exp(-x) - x, init = 0, max_iter=100, erro_tol=1e-5)

Valor dentro do treshold de erro


0.56714471334657

In [29]:
ponto_fixo(f = lambda x : np.exp(-x) - x, init = 0, max_iter=7)

Valor dentro do treshold de erro


0.5796123355033789

- Newton

In [ ]:
def newton_raphson(f, max_iter = 100):
    